In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1838,tt0218967,60000000,124745083,The Family Man,Nicolas Cage|TÃ©a Leoni|Don Cheadle|Saul Rubin...,Brett Ratner,What if you made different choices? What if yo...,"Jack's lavish, fast-paced lifestyle changes on...",125,Comedy|Drama|Romance|Fantasy,Universal Pictures,12/12/2000,6.4,2000
903,tt0278504,46000000,113714830,Insomnia,Al Pacino|Robin Williams|Maura Tierney|Hilary ...,Christopher Nolan,A tough cop. A brilliant killer. An unspeakabl...,Two Los Angeles homicide detectives are dispat...,118,Crime|Mystery|Thriller,Section Eight|Alcon Entertainment|Witt/Thomas ...,5/24/2002,6.6,2002
1580,tt0319262,125000000,544272402,The Day After Tomorrow,Dennis Quaid|Jake Gyllenhaal|Emmy Rossum|Dash ...,Roland Emmerich,Where will you be?,"After years of unabated global warming, the gr...",124,Action|Adventure|Science Fiction|Thriller,Lions Gate Films|Twentieth Century Fox Film Co...,5/26/2004,6.1,2004
1303,tt1869716,6500000,2401510,The East,Brit Marling|Alexander SkarsgÃ¥rd|Ellen Page|T...,Zal Batmanglij,"Spy on us, we'll spy on you.",An operative for an elite private intelligence...,116,Drama|Thriller,Scott Free Productions,5/31/2013,6.3,2013
1080,tt2215719,12000000,32726956,Katy Perry: Part of Me,Katy Perry|Russell Brand|Shannon Woodward|Rach...,Dan Cutforth|Jane Lipsitz,Be yourself and you can be anything,Giving fans unprecedented access to the real l...,93,Documentary|Music,Paramount Pictures|Imagine Entertainment|MTV F...,6/28/2012,6.5,2012


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

In [5]:

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
data['release_date'] = pd.to_datetime(data['release_date'], format='%m/%d/%Y')
data['profit'] = data['revenue'] - data['budget']
data['release_month'] = data['release_date'].dt.month
data['len_title'] = data.original_title.str.len()

In [6]:
data['count_title_word'] = data.overview.str.split().str.len()

In [7]:
def dict_df(data, column, sep = '|'):
    """Формируем новый dataframe по столбцу с перечислением"""
    df =pd.DataFrame(columns=['imdb_id', column])
    for row in data.itertuples():
        column_value_list = getattr(row, column).split(sep)
        for column_value in column_value_list:
            df.loc[len(df)] = [row.imdb_id, column_value]
    return df

In [8]:
genres = dict_df(data,'genres')
directors = dict_df(data,'director')
casts = dict_df(data,'cast')
productions = dict_df(data, 'production_companies')

In [9]:
"""Унифицируем название 'Warner Bros' в данных"""
uni_production_companies = (productions['production_companies']
                                .unique().tolist())
uni_production_companies = list(filter(lambda x: x.lower()
                                       .find('warner') != -1
                                        , uni_production_companies))
productions.loc[(productions['production_companies']
                 .isin(uni_production_companies)
                 , 'production_companies')] = 'Warner Bros'

In [10]:
RU_MONTH_VALUES = {
    'Январь': 1,
    'Февраль': 2,
    'Март': 3,
    'Апрель': 4,
    'Май': 5,
    'Июнь': 6,
    'Июль': 7,
    'Август': 8,
    'Сентябрь': 9,
    'Октябрь': 10,
    'Ноябрь': 11,
    'Декабрь': 12,
}


def int_value_to_ru_month(month):
    """Переводит номер месяца в русское название"""
    for k, v in RU_MONTH_VALUES.items():
        month = str(month).replace(str(v), k)
    return month

In [11]:
 def in_one_pr(list_value, element):
    """проверяет содержатся ли все элементы element в list_value"""
    for y in element:
        if  not y in list_value.values:
            return False
    return True

In [12]:
def cast_func(cast_list):
    """Разбивает актеров конкретного фильма по парам, оставляя уникальные пары"""
    cast_together = []
    for name_1 in cast_list:
        for name_2 in cast_list:
            if name_1 == name_2:
                continue
            cast_together += [[name_1, name_2]]
    return cast_together

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [13]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [14]:
# тут пишем ваш код для решения данного вопроса:
max_budget = data.budget.max()
max_budget_films = data[data.budget == max_budget]
answers['1'] = "{}. {} ({})".format(max_budget_films.index[0]
                  , max_budget_films.loc[max_budget_films.index[0]
                                         , 'original_title']
                  , max_budget_films.loc[max_budget_films.index[0]
                                         , 'imdb_id'])

# 2. Какой из фильмов самый длительный (в минутах)?

In [15]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '...'

In [16]:
max_runtime = data.runtime.max()
max_runtimes = data[data.runtime == max_runtime]
answers['2'] = "{}. {} ({})".format(max_runtimes.index[0]
                                    , max_runtimes.loc[max_runtimes.index[0]
                                                       , 'original_title']
                                    , max_runtimes.loc[max_runtimes.index[0]
                                                       , 'imdb_id'])

# 3. Какой из фильмов самый короткий (в минутах)?





In [17]:
min_runtime = data.runtime.min()
min_runtimes = data[data.runtime == min_runtime]
answers['3'] = ("{}. {} ({})".format(min_runtimes.index[0]
                                     , min_runtimes.loc[min_runtimes.index[0]
                                                        , 'original_title']
                                     , min_runtimes.loc[min_runtimes.index[0]
                                                        , 'imdb_id']))

# 4. Какова средняя длительность фильмов?


In [18]:
mean_runtime = data.runtime.mean()
answers['4'] = int(round(mean_runtime,0))

# 5. Каково медианное значение длительности фильмов? 

In [19]:
median_runtime = data.runtime.median()
answers['5'] = int(round(median_runtime))

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [20]:
# лучше код получения столбца profit вынести в Предобработку что в начале
max_profit = data.profit.max()
max_profit_films = data[data.profit == max_profit]
answers['6'] = "{}. {} ({})".format(max_profit_films.index[0]
                    , max_profit_films.loc[max_profit_films.index[0]
                                           , 'original_title']
                    , max_profit_films.loc[max_profit_films.index[0]
                                           , 'imdb_id'])

# 7. Какой фильм самый убыточный? 

In [21]:
min_profit = data.profit.min()
min_profit_films = data[data.profit == min_profit]
answers['7'] = "{}. {} ({})".format(min_profit_films.index[0]
                    , min_profit_films.loc[min_profit_films.index[0]
                                           , 'original_title']
                    , min_profit_films.loc[min_profit_films.index[0]
                                           , 'imdb_id'])

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
profits = data[data.profit > 0]
answers['8'] = profits['original_title'].count()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
max_profit_2008 = data[data.release_year == 2008].profit.max()
max_profit_2008_films = data[(data.release_year == 2008)
                             & (data.profit == max_profit_2008)]
answers['9'] = "{}. {} ({})".format(max_profit_2008_films.index[0]
                 , max_profit_2008_films.loc[max_profit_2008_films.index[0]
                                             , 'original_title']
                 , max_profit_2008_films.loc[max_profit_2008_films.index[0]
                                             , 'imdb_id'])

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [24]:
min_profit_12_14 = data[(data.release_year >= 2012) 
                        & (data.release_year <= 2014)].profit.min()
min_profit_12_14_films = data[(data.release_year >= 2012) 
                              & (data.release_year <= 2014) 
                              & (data.profit == min_profit_12_14)]
answers['10'] = "{}. {} ({})".format(min_profit_12_14_films.index[0]
                   , min_profit_12_14_films
                                     .loc[min_profit_12_14_films.index[0]
                                          , 'original_title']
                   , min_profit_12_14_films
                                     .loc[min_profit_12_14_films.index[0]
                                          , 'imdb_id'])

# 11. Какого жанра фильмов больше всего?

In [25]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
result = (genres.groupby('genres').count()
          .sort_values(by='imdb_id', ascending = False)
          .head(1).reset_index()
         )
answers['11'] = result.loc[0,'genres']

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [26]:
merged_profit = data[data.profit>0].merge(genres, on = 'imdb_id'
                                          , how = 'inner'
                                          , suffixes=('_left',''))
result = (merged_profit.groupby('genres').count()
          .sort_values(by='imdb_id', ascending = False)
          .head(1).reset_index()
         )
answers['12'] = result.loc[0,'genres']

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [27]:
merged_director = data.merge(directors, on = 'imdb_id'
                             , how = 'inner'
                             , suffixes=('_left',''))
result = (merged_director.groupby('director').sum()
          .sort_values(by='profit', ascending = False)
          .head(1).reset_index()
         )
answers['13'] = result.loc[0,'director']

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [28]:
merged_genre_director =  directors.merge(genres, on = 'imdb_id'
                                         , how = 'inner'
                                         , suffixes=('_left',''))
result = (merged_genre_director[merged_genre_director['genres'] == 'Action']
            .groupby('director').count()
              .sort_values(by='imdb_id', ascending = False)
                  .head(1).reset_index()
         )
answers['14'] = result.loc[0,'director']

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [29]:
merged_cast = data.merge(casts, on = 'imdb_id'
                         , how = 'inner'
                         , suffixes=('_left',''))

In [30]:
result = (merged_cast[merged_cast['release_year']== 2012]
            .groupby('cast')[['revenue']].sum()
              .sort_values(by='revenue', ascending = False)
                  .head(1).reset_index()
         )
answers['15'] = result.loc[0,'cast']

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [31]:
mean_budget = data.budget.mean()
result = (merged_cast[merged_cast['budget']> mean_budget]
          .groupby('cast').count()
              .sort_values(by='imdb_id', ascending = False)
                  .head(1).reset_index()
         )
answers['16'] = result.loc[0,'cast']

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [32]:
merged_genre_director =  casts.merge(genres, on = 'imdb_id'
                                     , how = 'inner'
                                     , suffixes=('_left',''))
result = (merged_genre_director[merged_genre_director['cast'] == 'Nicolas Cage']
            .groupby('genres').count()
                .sort_values(by='imdb_id', ascending = False)
                  .head(1).reset_index()
         )
answers['17'] = result.loc[0,'genres']

# 18. Самый убыточный фильм от Paramount Pictures

In [33]:
merge_productions = data.merge(productions, on = 'imdb_id'
                               , how = 'inner'
                               , suffixes=('_left',''))

In [34]:
result = (merge_productions[(merge_productions['production_companies'] 
                                == 'Paramount Pictures')]
            .sort_values(by = 'profit').head(1)
         )
answers['18'] = "{} ({})".format(result.loc[result.index[0]
                                            , 'original_title']
                                 ,result.loc[result.index[0]
                                             , 'imdb_id'])

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [35]:
result = (data.groupby('release_year').sum()
            .sort_values(by ='revenue', ascending = False)
                  .head(1).reset_index()
         )
answers['19'] = result.loc[0,'release_year']

# 20. Какой самый прибыльный год для студии Warner Bros?

In [36]:
result = (merge_productions[(merge_productions['production_companies']
                                == 'Warner Bros')]
            .groupby('release_year').sum()
                .sort_values(by ='revenue', ascending = False)
                  .head(1).reset_index()
         )
answers['20'] = result.loc[0,'release_year']

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [37]:
result = (data.groupby('release_month').count()
              .sort_values(by ='imdb_id', ascending = False)
                  .head(1).reset_index()
         )
answers['21'] = int_value_to_ru_month(result.loc[0,'release_month'])

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [38]:
result = data[data['release_month'].isin([6,7,8])].count()
result
answers['22'] = result['imdb_id']

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [39]:
result = (merged_director[merged_director['release_month'].isin([12,1,2])]
            .groupby('director').count()
              .sort_values(by ='imdb_id', ascending = False)
                  .head(1).reset_index()
         )
answers['23'] = result.loc[0,'director']

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [40]:
result = (merge_productions.groupby('production_companies').mean()
              .sort_values(by='len_title', ascending = False)
                  .head(1).reset_index()
         )
answers['24'] = result.loc[0,'production_companies']

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [41]:
result = (merge_productions.groupby('production_companies').mean()
              .sort_values(by='count_title_word', ascending = False)
                  .head(1).reset_index()
         )
answers['25'] = result.loc[0,'production_companies']

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [42]:
one_of = [['Inside Out', 'The Dark Knight', '12 Years a Slave']
          , ['BloodRayne'
             , 'The Adventures of Rocky & Bullwinkle Batman Begins'
             , 'The Lord of the Rings: The Return of the King']
          , ['Upside Down 300'
             , 'Lucky Number Slevin, Kill Bill: Vol. 1 Upside Down']
          , ['Inside Out', 'Iron Man']]
p = np.percentile(data['vote_average'], 99)
result = data[data['vote_average']>p]
answers['26'] = ', '.join([x for x in one_of 
                            if in_one_pr(result['original_title'], x)][0])

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [43]:
one_of = [['Johnny Depp', 'Helena Bonham Carter']
          , ['Ben Stiller', 'Owen Wilson']
          , ['Vin Diesel', 'Paul Walker']
          , ['Adam Sandler', 'Kevin James']
          , ['Daniel Radcliffe', 'Rupert Grint']]
new_data_cast = data.copy()
new_data_cast.cast = new_data_cast.cast.str.split('|')
new_data_cast.cast = new_data_cast.cast.apply(cast_func)
new_data_cast = new_data_cast.explode('cast')
counts_pair = new_data_cast.cast.value_counts()
max_val = counts_pair.max()
counts_pair = counts_pair[counts_pair == max_val]
for index, value in counts_pair.items():
    if index in one_of:
        answers['27'] = ' & '.join(index)

# Submission

In [44]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [45]:
# и убедиться что ни чего не пропустил)
len(answers)

27